In [29]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Embedding, Concatenate
from tensorflow.keras.models import Model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

dataframe = pd.read_csv('data/processed.csv')

dataframe = dataframe.sample(frac=0.1, random_state=1)

print("Shape of dataframe: ", dataframe.shape)
dataframe.head()

Shape of dataframe:  (103113, 8)


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
887470,4556,8,Abduction,Robin Cook,2000,Berkley Publishing Group,"south berwick, maine, usa",34
56511,234623,0,Nancy Kerrigan: In My Own Words,Nancy Kerrigan,1996,Disney Pr (Jp),"scarborough, maine, usa",34
238538,190925,0,Bad Girl : A Novel,MICHELE JAFFE,2003,Ballantine Books,"hobe sound, florida, usa",51
610781,56661,0,The Joy Luck Club,Amy Tan,1994,Prentice Hall (K-12),"green, ohio, usa",34
179676,56856,0,The Pleasure of My Company: A Novel,Steve Martin,2003,Hyperion,"escondido, california, usa",23


In [30]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(dataframe['Book-Title'])
X_train, X_test = train_test_split(X, test_size=0.2, random_state=1)

In [31]:
userInput = Input(shape=(1,))
userEmbedding = Embedding(input_dim=263407, output_dim=32)(userInput)
userVector = Flatten()(userEmbedding)
userOutput = Dense(1)(userVector)

bookInput = Input(shape=(1,))
bookEmbedding = Embedding(input_dim=270713, output_dim=32)(bookInput)
bookVector = Flatten()(bookEmbedding)
bookOutput = Dense(1)(bookVector)

concatenate = Concatenate()([userVector, bookVector])

dense = Dense(128, activation='relu')(concatenate)
output = Dense(1)(dense)

In [32]:
model = Model([userInput, bookInput], output)
model.compile(loss='mean_squared_error', optimizer='adam')

early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model.fit([dataframe['User-ID'], X_train], dataframe['Book-Rating'], epochs=10, verbose=1, callbacks=[early_stopping])

ValueError: Data cardinality is ambiguous:
  x sizes: 103113, 82490
  y sizes: 103113
Make sure all arrays contain the same number of samples.